In [8]:
from midi.utils import midiread, midiwrite
import os
import numpy as np

In [5]:
midi_range = (21,109)
dt = 0.3

data_dir = '../rnnrbm/data/Nottingham/train/'

files = os.listdir(data_dir)

files = [data_dir + f for f in files if '.mid' in f]

dataset = [midiread(f, midi_range, dt).piano_roll for f in files]

In [40]:
np.shape(dataset[5])

(200, 88)

In [42]:
max_notes = 16*4
step = 9

In [43]:
input_measures = []
next_notes = []

for song in dataset:
    for i in range(0, len(song) - max_notes, step):
        input_measures.append(song[i: i + max_notes])
        next_notes.append(song[i + max_notes])

In [73]:
X = np.zeros((11000, 64, 88), dtype=np.bool)
y = np.zeros((11000, 88), dtype=np.bool)

In [77]:
type(X)

numpy.ndarray

In [78]:
type(input_measures)

list

In [80]:
X = np.array(input_measures)

In [81]:
y = np.array(next_notes)

In [114]:
seed_song = random.randint(0, len(dataset))
seed = np.array(dataset[seed_song][0:64])

In [118]:
random.randint(0, len(dataset))

641

In [166]:
seed.shape

(64, 88)

In [174]:
a = []

In [175]:
for note in seed:
    a.append(note)

In [122]:
gen = model.predict(seed[np.newaxis,:])

In [133]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    print(np.shape(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [129]:
np.shape(gen)

(1, 88)

In [165]:
np.random.binomial(n=1, p=gen[0], size=gen[0].shape)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [135]:
sample(gen[0], 1)

(88,)


29

In [136]:
def noteIdx_to_oneHot(idx, r):
    onehot = np.zeros(r[1] - r[0])
    onehot[idx - r[0]] = 1
    return onehot

In [141]:
noteIdx_to_oneHot(108, (21, 109))

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])

In [91]:
next_note = seed[0]

In [97]:
np.vstack((seed[1:],next_note)).shape

(64, 88)

In [98]:
a = []

In [101]:
a.append(next_note)

In [102]:
np.shape(a)

(2, 88)

In [105]:
midiwrite('test_output.mid', a, (21,109), 0.3)

In [57]:
np.shape(X)

(200294, 20, 59)

In [58]:
np.shape(y)

(200294, 59)

In [62]:
midi_range[1] - midi_range[0]

88

In [65]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=False, input_shape=(max_notes, midi_range[1] - midi_range[0])))
model.add(Dropout(0.2))
model.add(Dense(midi_range[1] - midi_range[0]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

Build model...


In [82]:
model.fit(X, y, batch_size=128, nb_epoch=1)

Epoch 1/1
 1536/11681 [==>...........................] - ETA: 268s - loss: 13.5307

KeyboardInterrupt: 

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [25]:
sentences[3]

'\nsupposing that trut'

In [22]:
next_chars[2]

'r'

In [46]:
np.shape(sentences)

(200294,)

In [51]:
np.shape(y)

(200294, 59)

In [64]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X[:256], y[:256], batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

        print()

Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
256/256 [==============================] - 5s - loss: 3.9551     

----- diversity: 0.2
----- Generating with seed: "ys, the evidence
of "
ys, the evidence
of 

----- diversity: 0.5
----- Generating with seed: "ys, the evidence
of "
ys, the evidence
of 

----- diversity: 1.0
----- Generating with seed: "ys, the evidence
of "
ys, the evidence
of 

----- diversity: 1.2
----- Generating with seed: "ys, the evidence
of "
ys, the evidence
of 

--------------------------------------------------
Iteration 2
Epoch 1/1
256/256 [==============================] - 6s - loss: 3.2254     

----- diversity: 0.2
----- Generating with seed: "eastful systems of w"
eastful systems of w

KeyboardInterrupt: 

In [60]:
path = 'nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 600901
total chars: 59
nb sequences: 200294
Vectorization...
